In [2]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# LSTM初始化

# Data

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Normalize your data
# This step is optional and depends on the nature of your dataset
from sklearn.preprocessing import StandardScaler
scaler_target = StandardScaler()
scaler_history = StandardScaler()  # Define scaler_history object

# load data
history = pd.read_csv('../data/results/history.csv', index_col='time',parse_dates=True)
ssp245 = pd.read_csv('../data/results/ssp245.csv', index_col=0)
ssp585 = pd.read_csv('../data/results/ssp585.csv', index_col=0)
history_wl = pd.read_csv('../data/results/history_wl.csv', index_col='time',parse_dates=True)

# Convert history data to scaled values
history_scaled = scaler_history.fit_transform(history)

target_scaled = scaler_target.fit_transform(history_wl)

# Convert to PyTorch tensors
history_tensor = torch.Tensor(history_scaled)
target_tensor = torch.Tensor(target_scaled)

# Split the dataset into training and testing datasets
dataset = TensorDataset(history_tensor, target_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Define a DataLoader for training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

## Define the model

In [ ]:
# Define LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # predicting 1 value

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = self.fc(out[:, -1, :])
        return out

# Hyperparameters
input_size = history_tensor.shape[1]
hidden_size = 50
num_layers = 2

model = LSTM(input_size, hidden_size, num_layers).to(device)


# Define Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        # Reshape input tensor to [batch_size, sequence_length, n_features]
        inputs = inputs.reshape(-1, 1, input_size)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 2 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

## Evaluate model

In [ ]:
# Load test data
test_history_tensors = []
test_target_tensors = []
for inputs, targets in test_loader:
    test_history_tensors.append(inputs)
    test_target_tensors.append(targets)

test_history_tensor = torch.cat(test_history_tensors, dim=0)
test_target_tensor = torch.cat(test_target_tensors, dim=0)

# Make predictions on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Do not compute gradient to save memory
    test_predictions = model(test_history_tensor.reshape(-1, 1, input_size))

# Compute evaluation metrics
test_rmse = np.sqrt(mean_squared_error(test_target_tensor.numpy(), test_predictions.numpy()))
test_r2 = r2_score(test_target_tensor.numpy(), test_predictions.numpy())

print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test R2 score: {test_r2:.4f}")

In [42]:
# Now suppose you have two future datasets SSP245 and SSP585
# They should be PyTorch tensors and have the same shape as the input
SSP245_tensor = torch.Tensor(scaler_history.transform(ssp245))  # Replace this line with your code if necessary
SSP585_tensor = torch.Tensor(scaler_history.transform(ssp585))  # Replace this line with your code if necessary

# We can use the trained model to predict the future
SSP245_predictions = model(SSP245_tensor.to(device).reshape(-1, 1, input_size))
SSP585_predictions = model(SSP585_tensor.to(device).reshape(-1, 1, input_size))

# If your data was scaled, you might want to rescale the predictions back to the original scale
SSP245_predictions_rescaled = scaler_target.inverse_transform(SSP245_predictions.detach().numpy())
SSP585_predictions_rescaled = scaler_target.inverse_transform(SSP585_predictions.detach().numpy())

In [5]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Reshape input tensor to [batch_size, sequence_length, n_features]
            inputs = inputs.reshape(-1, 1, input_size)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch+1) % 2 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


def tune_parameters(hidden_sizes, num_layers_list, learning_rates, num_epochs=10):
    best_rmse = float('inf')
    best_r2 = float('-inf')
    best_params = None

    for hidden_size in hidden_sizes:
        for num_layers in num_layers_list:
            for lr in learning_rates:
                print(f"Training model with hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={lr}")
                model = LSTM(input_size, hidden_size, num_layers).to(device)
                if torch.cuda.device_count() > 1:
                    print("Let's use", torch.cuda.device_count(), "GPUs!")
                    model = nn.DataParallel(model)
                criterion = nn.MSELoss()
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs)

                # Evaluate the model on the test set
                model.eval()  # Set the model to evaluation mode
                with torch.no_grad():  # Do not compute gradient to save memory
                    test_predictions = model(test_history_tensor.reshape(-1, 1, input_size))

                # Compute evaluation metrics
                test_rmse = np.sqrt(mean_squared_error(test_target_tensor.cpu().numpy(), test_predictions.cpu().numpy()))
                test_r2 = r2_score(test_target_tensor.cpu().numpy(), test_predictions.cpu().numpy())

                print(f"Test RMSE: {test_rmse:.4f}, Test R2 score: {test_r2:.4f}")

                # Update best params if current model is better
                if test_rmse < best_rmse and test_r2 > best_r2:
                    best_rmse = test_rmse
                    best_r2 = test_r2
                    best_params = (hidden_size, num_layers, lr)

    print(f"Best parameters are hidden_size={best_params[0]}, num_layers={best_params[1]}, learning_rate={best_params[2]}")
    print(f"Best test RMSE: {best_rmse:.4f}, Best test R2 score: {best_r2:.4f}")


hidden_sizes = [50, 100, 150]
num_layers_list = [1, 2, 3, 4]
learning_rates = [0.1, 0.01, 0.001]
tune_parameters(hidden_sizes, num_layers_list, learning_rates, num_epochs=10)


Training model with hidden_size=50, num_layers=1, learning_rate=0.1
Let's use 4 GPUs!
Epoch [2/10], Loss: 0.3021
Epoch [4/10], Loss: 0.4437
Epoch [6/10], Loss: 0.4912
Epoch [8/10], Loss: 0.4524
Epoch [10/10], Loss: 0.4505
Test RMSE: 0.8381, Test R2 score: 0.3031
Training model with hidden_size=50, num_layers=1, learning_rate=0.01
Let's use 4 GPUs!
Epoch [2/10], Loss: 0.2752
Epoch [4/10], Loss: 0.2998
Epoch [6/10], Loss: 0.3211
Epoch [8/10], Loss: 0.3352
Epoch [10/10], Loss: 0.3393
Test RMSE: 0.7732, Test R2 score: 0.4070
Training model with hidden_size=50, num_layers=1, learning_rate=0.001
Let's use 4 GPUs!
Epoch [2/10], Loss: 0.2836
Epoch [4/10], Loss: 0.2556
Epoch [6/10], Loss: 0.2567
Epoch [8/10], Loss: 0.2605
Epoch [10/10], Loss: 0.2631
Test RMSE: 0.7959, Test R2 score: 0.3716
Training model with hidden_size=50, num_layers=2, learning_rate=0.1
Let's use 4 GPUs!
Epoch [2/10], Loss: 0.4066
Epoch [4/10], Loss: 0.3115
Epoch [6/10], Loss: 0.4881
Epoch [8/10], Loss: 0.2967
Epoch [10/10],